In [1]:
import os
import numpy as np
import json

import nltk
nltk.download('punkt')

from nltk import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
en_stopwords = stopwords.words('english')  + ['shh', 'hoo', 'boo', 'uhoh', 'aah', 'heh', 'huh', 'ooh', 'yo']

[nltk_data] Downloading package punkt to /home/angelo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
path = "/home/angelo/Desktop/master_offline/progettone_github/Animation-studios-analysis/sottotitoli/clean_subtitles/Disney/Animation/"

all_text = []
for r, d, files in os.walk(path):
    for f in files:
        filename = r+'/'+f
        with open(filename, 'r', encoding ='utf-8', errors='ignore') as fh:
            text = fh.read()
            all_text.append(text)

In [3]:
len(all_text)

88

# sentence splitting

In [4]:
one_document = ' '.join(all_text).lower()
one_document = one_document.replace('\n', ' ')
sentences = sent_tokenize(one_document)

In [5]:
len(sentences)

111766

In [6]:
sentences[0]

'long ago, in the faraway land of ancient greece, there was a golden age of powerful gods and extraordinary heroes.'

In [7]:
tok_sents = [[word for word in word_tokenize(sentence) if (word.isalpha())] for sentence in sentences]

In [8]:
tok_sents[0]

['long',
 'ago',
 'in',
 'the',
 'faraway',
 'land',
 'of',
 'ancient',
 'greece',
 'there',
 'was',
 'a',
 'golden',
 'age',
 'of',
 'powerful',
 'gods',
 'and',
 'extraordinary',
 'heroes']

In [9]:
tok_sents[0:3]

[['long',
  'ago',
  'in',
  'the',
  'faraway',
  'land',
  'of',
  'ancient',
  'greece',
  'there',
  'was',
  'a',
  'golden',
  'age',
  'of',
  'powerful',
  'gods',
  'and',
  'extraordinary',
  'heroes'],
 ['and',
  'the',
  'greatest',
  'and',
  'strongest',
  'of',
  'all',
  'these',
  'heroes',
  'was',
  'the',
  'mighty',
  'hercules'],
 ['but', 'what', 'is', 'the', 'measure', 'of', 'a', 'true', 'hero']]

In [10]:
from collections import Counter
counter = Counter([len(s) for s in sentences])
sorted(counter.items())

[(1, 12),
 (2, 18),
 (3, 2705),
 (4, 3954),
 (5, 5501),
 (6, 2962),
 (7, 3170),
 (8, 3179),
 (9, 3019),
 (10, 3871),
 (11, 3325),
 (12, 2894),
 (13, 3187),
 (14, 3268),
 (15, 3041),
 (16, 3059),
 (17, 3018),
 (18, 2894),
 (19, 3038),
 (20, 2616),
 (21, 2551),
 (22, 2410),
 (23, 2474),
 (24, 2262),
 (25, 2126),
 (26, 2046),
 (27, 2050),
 (28, 1838),
 (29, 1760),
 (30, 1680),
 (31, 1607),
 (32, 1473),
 (33, 1411),
 (34, 1392),
 (35, 1291),
 (36, 1160),
 (37, 1097),
 (38, 1077),
 (39, 989),
 (40, 923),
 (41, 894),
 (42, 854),
 (43, 787),
 (44, 727),
 (45, 677),
 (46, 721),
 (47, 675),
 (48, 562),
 (49, 549),
 (50, 538),
 (51, 533),
 (52, 483),
 (53, 464),
 (54, 438),
 (55, 454),
 (56, 440),
 (57, 372),
 (58, 369),
 (59, 354),
 (60, 314),
 (61, 337),
 (62, 282),
 (63, 266),
 (64, 275),
 (65, 257),
 (66, 245),
 (67, 248),
 (68, 217),
 (69, 211),
 (70, 233),
 (71, 216),
 (72, 229),
 (73, 203),
 (74, 183),
 (75, 174),
 (76, 184),
 (77, 137),
 (78, 138),
 (79, 136),
 (80, 113),
 (81, 133),
 (8

In [11]:
[s for s in sentences if len(s) == 908]

["you're jokin'  you're jokin'  i can't believe  my eyes  you're jokin' me  you gotta be  this can't be  the right guy  he's ancient  he's ugly  i don't know  which is worse  i might just split a seam now  if i don't die laughin' first  mr. oogie boogie says  there's trouble close at hand  you better  pay attention now  'cause i'm  the boogie man  and if you  aren't shakin'  there's somethin'  very wrong  'cause this may be the last time  you hear the boogie song  whoaoh  wahah  whoaho  ow  whoaoh  wow  i'm the oogie boogie man  release me now or you must  face the dire consequences  the children are expecting me  so please, come to your senses  ah, you're jokin'  you're jokin'  i can't believe  my ears  would someone  shut this fella up  i'm drownin'  in my tears  he's funny  i'm laughin'  you really are  too much  and now  with your permission  i'm goin' to do my stuff  what are you going to do?"]

# WORD2VEC

In [12]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

In [13]:
from gensim.models import Word2Vec
w2v_model = Word2Vec(tok_sents, size=200, window=10, min_count=15, sg=0, iter=20, negative=10)

In [14]:
#learned vocabulary
words = list(w2v_model.wv.vocab)
len(words)

2725

In [15]:
w2v_model.wv.most_similar(['aurora'])

[('hail', 0.8734434247016907),
 ('atta', 0.7436292767524719),
 ('princess', 0.7379677891731262),
 ('queen', 0.7312297224998474),
 ('health', 0.7117342352867126),
 ('gift', 0.6559451818466187),
 ('beloved', 0.6418209671974182),
 ('yudhishtir', 0.610481858253479),
 ('arendelle', 0.6002275943756104),
 ('king', 0.59516841173172)]

In [16]:
w2v_model.wv.most_similar(['king', 'princess'], ['prince'])

[('queen', 0.6415515542030334),
 ('aurora', 0.6103020310401917),
 ('hail', 0.5977053642272949),
 ('england', 0.5353997349739075),
 ('gods', 0.5154109597206116),
 ('kingdom', 0.5104103088378906),
 ('gift', 0.4848730266094208),
 ('health', 0.47159403562545776),
 ('racer', 0.4565645456314087),
 ('arendelle', 0.4518205523490906)]

In [17]:
w2v_model.wv.similarity('king', 'majesty')

0.39351386

# constructin matrix for graph

In [18]:
voc = set(w2v_model.wv.vocab)
len(voc)

2725

In [19]:
voc = voc.difference(en_stopwords)
len(voc)

2591

In [20]:
indici = dict()
for index, key in enumerate(voc):
    if key not in indici:
        indici[key] = index
        
reverse_indici = dict()
for term, termID in indici.items():
    reverse_indici[termID] = term

In [21]:
#costruzione matrice
graph_matrix = np.zeros(shape=(len(voc), len(voc)), dtype=np.float32)

for index1, key1 in enumerate(voc):
    for index2, key2 in enumerate(voc):
        graph_matrix[index1 ][index2] = w2v_model.wv.similarity(key1, key2)
        
len(graph_matrix)

2591

In [35]:
#constructing the list of tuples
graph_list = []
nodes = set() # <-- NUOVO
for x in range(len(graph_matrix)):
    for y in range(x+1, len(graph_matrix)):
        if graph_matrix[x][y] >= 0.7 and x != y:
            nodes |= set([reverse_indici[x], reverse_indici[y]])
            graph_list.append((reverse_indici[x],reverse_indici[y],graph_matrix[x][y]))      
print('nodes:', len(nodes))
print('links:', len(graph_list))

nodes: 279
links: 253


In [36]:
# NUOVO
graph = dict()
graph["nodes"] = [{"id": t} for t in nodes]
graph["links"] = [{"source":t[0], "target":t[1], "value":float(t[2])} for t in graph_list]
with open('data.json', 'w') as outfile:
    json.dump(graph, outfile)

## FastText

In [24]:
from gensim.models import FastText
fast_model = FastText(tok_sents,size=100, window=10, min_count=5, sg=1, iter=20, negative=10)

In [25]:
fast_model.wv.most_similar(['apple'])

[('applause', 0.6684627532958984),
 ('example', 0.6109883785247803),
 ('triple', 0.6024612784385681),
 ('butch', 0.5653321146965027),
 ('pie', 0.5465620160102844),
 ('purple', 0.5435370206832886),
 ('profile', 0.5335412621498108),
 ('juicy', 0.532211184501648),
 ('diet', 0.5283253788948059),
 ('dimple', 0.526831328868866)]

In [26]:
fast_model.wv.most_similar(['king'])

[('richard', 0.7180807590484619),
 ('viking', 0.6615813970565796),
 ('hastinapur', 0.6610158681869507),
 ('kingdom', 0.644915759563446),
 ('england', 0.6400753855705261),
 ('arthur', 0.6266749501228333),
 ('stefan', 0.5838980674743652),
 ('horned', 0.581543505191803),
 ('yudhishtir', 0.571908175945282),
 ('skeleton', 0.5635082125663757)]

In [27]:
fast_model.wv.most_similar(['mirror'])

[('symbol', 0.5950874090194702),
 ('thunderbolt', 0.5921159386634827),
 ('tragic', 0.5835063457489014),
 ('hideout', 0.5819932818412781),
 ('newspaper', 0.5803345441818237),
 ('dinosaur', 0.5791431069374084),
 ('greeneyed', 0.5747506618499756),
 ('triumph', 0.5740880370140076),
 ('wondrous', 0.5735598206520081),
 ('oneeyed', 0.5681384205818176)]

In [28]:
fasttext_wAp_analogy = fast_model.wv.evaluate_word_analogies(test_file, dummy4unknown=True)

NameError: name 'test_file' is not defined

# google precomputed word embeddings

In [ ]:
import gensim
google_w2v_file = 'GoogleNews-vectors-negative300.bin'
google_model = gensim.models.KeyedVectors.load_word2vec_format(google_w2v_file,binary=True)

In [ ]:
google_model.most_similar(['king','female'],['male'])

In [ ]:
w2v_large_analogy = google_model.wv.accuracy(test_file)